Complete the exercises below For **Assignment #11**.

For this assignment we'll use the [California Housing Data](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html).

This data is available in the [Scikit-Learn](https://scikit-learn.org/stable/index.html) Python module.

To retrieve the data,
1. open a new Python notebook with the `data730` kernel, and,
2. run the following code.

```python
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame=True)

(
    california_housing.data
    .assign(MedianHouseValue=california_housing.target)
    .to_csv('california-housing.csv', index=False)
)
```

This will produce a new file called `california-housing.csv` that we'll import in this notebook.

Back in the Python notebook, you can use the following snippet to get a description of the dataset.

```python
print(california_housing.DESCR)
```


Load the `tidymodels` package.

In [6]:
library('tidymodels')



Warning message:
“Your system is mis-configured: ‘/var/db/timezone/localtime’ is not a symlink”
Warning message:
“‘/var/db/timezone/localtime’ is not identical to any known timezone file”
── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tibble       3.2.1
✔ ggplot2      3.5.1     ✔ tidyr        1.3.1
✔ infer        1.0.7     ✔ tune         1.2.1
✔ modeldata    1.4.0     ✔ workflows    1.1.4
✔ parsnip      1.2.1     ✔ workflowsets 1.1.0
✔ purrr        1.0.2     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()
• Dig deeper into tidy modeling with R at https://www.tmwr.org



Run the following cell to read in our data.

In [7]:
housing = readr::read_csv('california-housing.csv', col_types = readr::cols())

housing |> head()

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8.3252,41,6.984127,1.0238095,322,2.555556,37.88,-122.23,4.526
8.3014,21,6.238137,0.9718805,2401,2.109842,37.86,-122.22,3.585
7.2574,52,8.288136,1.0734463,496,2.802260,37.85,-122.24,3.521
5.6431,52,5.817352,1.0730594,558,2.547945,37.85,-122.25,3.413
3.8462,52,6.281853,1.0810811,565,2.181467,37.85,-122.25,3.422
4.0368,52,4.761658,1.1036269,413,2.139896,37.85,-122.25,2.697


Let's build a random forest model to predict `MedianHouseValue`.

1. Create your model, set the engine to `ranger` with `importance = 'impurity'` and the mode as `regression`.
2. Fit the model using `MedianHouseValue` as the response variable and all other columns as predictors.

In [8]:
library('dplyr')

rf_model <- rand_forest(
    mode = "regression",
    trees = 1000,
    mtry = 5,
    min_n = 10
) %>%
    set_engine("ranger", importance = "impurity")




In [9]:
rf_fit <- rf_model %>%
    fit(MedianHouseValue ~ ., data = housing)


In [10]:
rf_fit


parsnip model object

Ranger result

Call:
 ranger::ranger(x = maybe_data_frame(x), y = y, mtry = min_cols(~5,      x), num.trees = ~1000, min.node.size = min_rows(~10, x),      importance = ~"impurity", num.threads = 1, verbose = FALSE,      seed = sample.int(10^5, 1)) 

Type:                             Regression 
Number of trees:                  1000 
Sample size:                      20640 
Number of independent variables:  8 
Mtry:                             5 
Target node size:                 10 
Variable importance mode:         impurity 
Splitrule:                        variance 
OOB prediction error (MSE):       0.2381874 
R squared (OOB):                  0.8211289 

❓ What is the OOB error for this model?

**Answer:**

In [11]:

rf_model <- rand_forest(
    mode = "regression",
    trees = 1000,
    mtry = 5,
    min_n = 10
) %>%
    set_engine("ranger", importance = "impurity", keep.inbag = TRUE)



In [12]:
rf_fit <- rf_model %>%
    fit(MedianHouseValue ~ ., data = housing)

rf_fit$fit


Ranger result

Call:
 ranger::ranger(x = maybe_data_frame(x), y = y, mtry = min_cols(~5,      x), num.trees = ~1000, min.node.size = min_rows(~10, x),      importance = ~"impurity", keep.inbag = ~TRUE, num.threads = 1,      verbose = FALSE, seed = sample.int(10^5, 1)) 

Type:                             Regression 
Number of trees:                  1000 
Sample size:                      20640 
Number of independent variables:  8 
Mtry:                             5 
Target node size:                 10 
Variable importance mode:         impurity 
Splitrule:                        variance 
OOB prediction error (MSE):       0.238166 
R squared (OOB):                  0.821145 

The OOB error is a measure of prediction accuracy based on samples that were not used to build each individual tree. It serves as an unbiased estimate of model performance.

Plot the feature importances using the `vip` package. Remember that you can use the `extract_fit_engine` function to get the `ranger` model object to send to the `vip` function.

In [13]:
install.packages('vip')


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [ ]:
library('vip')

rf_model <- rand_forest(
    mode = "regression",
    trees = 1000,
    mtry = 5,
    min_n = 10
) %>%
    set_engine("ranger", importance = "impurity")

rf_fit <- rf_model %>%
    fit(MedianHouseValue ~ ., data = housing)




Attaching package: ‘vip’


The following object is masked from ‘package:utils’:

    vi




In [ ]:
ranger_model <- extract_fit_engine(rf_fit)

vip(ranger_model)

vip(ranger_model) +
    ggtitle("Feature Importances for Random Forest Model") +
    theme_minimal()


It looks like `Latitude` and `Longitude` are important for predicting home value. Let's explore those variables.

Install the `mapdata` package into your `data730` environment.

1. Open a Terminal, and,
2. Run the following command:

```bash
conda install -c conda-forge -n data730 r-mapdata
```

Load the `mapdata` package.

In [ ]:
library('mapdata')



The following code will generate a map of California. Adapt this code to plot `Latitude` and `Longitude` from the `housing` dataset, color points by the `MedianHouseValue`. [Here is an example for reference](https://raw.githubusercontent.com/UNC-DATA-730/a10/main/example.png).


```r
california = map_data("state") |> filter(region == 'california')

ggplot(data = california) + 
    geom_polygon(aes(x = long, y = lat), fill = "grey90", color = "grey40") +
    coord_quickmap() +
    theme_void()
```

In [ ]:

library('ggplot2')


In [ ]:

california <- map_data("state") %>%
  filter(region == "california")


In [ ]:

ggplot(data = california) + 
  geom_polygon(aes(x = long, y = lat), fill = "grey90", color = "grey40") +
  geom_point(data = housing, 
             aes(x = Longitude, y = Latitude, color = MedianHouseValue), 
             size = 1.5, alpha = 0.7) +
  scale_color_viridis_c(option = "plasma") + 
  coord_quickmap() +
  theme_void() +
  labs(color = "Median House Value", 
       title = "California Housing Prices",
       x = "Longitude",
       y = "Latitude")
